# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import json
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from utils.print_utils import printProgress, printSuccess, printWarning
from coastsat import NOC_tools, NOC_shoreline, NOC_download, SDS_tools, SDS_shoreline
        
# directory where the data will be accessed and stored
data_partition = 'c:\\data\\coastsat'
country = 'UK'
base_dir_path = os.path.join(data_partition, country)  
sites_dir_path = os.path.join(base_dir_path, 'sites')

sites = os.listdir(sites_dir_path)

settings ={
#           'results_dir_path': results_dir_path,
#           'output_epsg': 32630, ## UK
#           'output_epsg': 32620, ## STV
#           'output_epsg': 32737, ## kenya
           'output_epsg': 22770, ## UK - BGS 1936
   
           'sat_name': 'S2',

           'cloud_thresh':0.9,
           
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
            'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
            'min_length_shoreline': 200,       # minimum length (in metres) of shoreline perimeter to be valid
            'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
 
            ## Image download Parameters\n",
            # Landsat\n",
            'LCloudScore': 15,         # Mean cloud score threshold (include images with less then threshold)\n",
            'add_L7_to_L5': True,      # Add Landsat 7 to Landsat 5 median composite if they are in same time period\n",
            'add_L5_to_L7': True,      # Add Landsat 5 to Landsat 7 median composite if they are in same time period\n",
            'add_L7_to_L8': False,      # Add Landsat 7 to Landsat 8 median composite if they are in same time period\n",
            'LCloudThreshold': 35,     # Pixels from a single image in a collection larger than this cloud score threshold\n",
                                       # will be masked.\n",
           
           'coregistration': False,
           
            # Sentinel
            'CLOUD_FILTER': 50,         # [Integer] Maximum image cloud cover percent allowed in image collection'
            'CLD_PRB_THRESH': 25,       # {Integer] Cloud probability (%); values greater than are considered cloud
            'NIR_DRK_THRESH': 0.08,     # [Float] Near-infrared reflectance; values less than are considered potential cloud shadow
            'CLD_PRJ_DIST': 2,          # [Float] Maximum distance (km) to search for cloud shadows from cloud edges |
            'BUFFER': 50,               # [Integer] Distance (m) to dilate the edge of cloud-identified objects |
   
           # labels for classes for the classifier and colours for display
           'classes':{'hard': [1, [1, 0, 0], 0], 'nature': [2, [0, 1, 0], 2],
                      'urban': [3, [1, 1, 0], 1], 'white-water': [4, [1, 0, 1], 4],
                      'water': [5, [0, 0.4, 1], 5], 'sand': [6, [1, 0.6, 0], 3]},
           
            'bands': {'L5': {'ms': [['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'BQA'], 30]},
                      'L8': {'pan': [['B8'],15],
                              'ms': [['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'BQA'], 30]},
                      'S2': {'10m': [['B2', 'B3', 'B4', 'B8'], 10],
                             '20m': [['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'], 20],
                             '60m': [['B12'], 60]}, },
            'SWIR': 'B12',
          }

dates = [
         ['2016-01-01', '2016-12-31'],
         ['2017-01-01', '2017-12-31'],
         ['2018-01-01', '2018-12-31'],
         ['2019-01-01', '2019-12-31'],
         ['2020-01-01', '2020-12-31'],
         ['2021-01-01', '2021-12-31']
        ]
download = False

## download optical data from GEE

In [ ]:
if download:
    
    for site in sites:       
        kml_filepath = os.path.join(sites_dir_path, site)
        kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
        roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)

        site_name = site[:site.find('.')]

        median_dir_path = os.path.join(base_dir_path, site_name, 'median')

        settings['median_dir_path'] = median_dir_path
        settings['polygon'] = roi_polygon
        settings['site_name'] = site_name

        for date_pair in dates:

            settings['dates'] = date_pair 

            printProgress(f'processing {site_name}: {date_pair}')

            NOC_download.retrieve_median_optical(settings)

        NOC_download.save_metadata(settings)

    print('FINISHED ...')

## find reference shoreline

In [ ]:
settings['date_range'] = ['2016-01-01', '2021-12-31']
settings['pansharpen'] = True
settings['max_dist_ref'] = 75
    
for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path
    
    printProgress(f'processing {site_name}')
    
    %matplotlib qt
    NOC_shoreline.find_reference_shoreline(settings) 

print('FINISHED ...')

## extract shorelines

In [ ]:
settings['date_range'] =['2016-01-01', '2021-12-31']
settings['pansharpen'] = True
#settings['reference_threshold'] = -0.416
settings['max_dist_ref'] = 100
batchProcess = True

for site in sites:
    
    site_name = site[:site.find('.')]

    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    results_dir_path = os.path.join(base_dir_path, site_name, 'results')
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path
    settings['results_dir_path'] = results_dir_path
    printProgress('')
        
    for date_pair in dates:
 
        settings['dates'] = date_pair 
    
        printProgress(f'processing {site_name}: {date_pair}')
        
        %matplotlib qt
        if settings['sat_name'] == 'S2':
            
            try:
                NOC_shoreline.extract_shoreline_optical(settings, batch=batchProcess)

            except Exception as inst:
                printWarning('skipping image - check date range')
                print(type(inst))
                print(inst.args)
                
        elif settings['sat_name'] in ['L5', 'L8']:
            
            metadata = NOC_download.load_metadata(settings)
            inputs ={ 'satname': settings['sat_name'],
                      'sitename': site_name,
                      'filepath': median_dir_path,
                    }
            settings['inputs'] = inputs
            print(settings['inputs'])
            SDS_shoreline.extract_shorelines(metadata, settings)

            
print('FINISHED ...')